In [0]:
import torch
import torch.nn as nn
from torchtext import data

torch.backends.cudnn.deterministic = True
TEXT = data.Field(tokenize = "spacy", include_lengths=True)
LABEL = data.LabelField(dtype = torch.float)

In [2]:
from torchtext import datasets
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

aclImdb_v1.tar.gz:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:03<00:00, 23.3MB/s]


In [3]:
print ('No. of training examples : {}'.format(len(train_data)))
print ('No. of testing examples : {}'.format(len(test_data)))

No. of training examples : 25000
No. of testing examples : 25000


In [0]:
train_data, valid_data = train_data.split()

In [5]:
print ('No. of training examples : {}'.format(len(train_data)))
print ('No. of validation examples : {}'.format(len(valid_data)))

No. of training examples : 17500
No. of validation examples : 7500


In [6]:
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE, vectors = "glove.6B.100d", unk_init = torch.Tensor.normal_)
LABEL.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [00:22, 39.2MB/s]                           
100%|█████████▉| 398407/400000 [00:17<00:00, 23150.02it/s]

In [0]:
batch_size = 64

device = torch.device('cuda')
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits((train_data, valid_data, test_data), batch_size = batch_size, sort_within_batch = True, device = device)

In [0]:
class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, pad_idx, no_of_layers = 2, bidirectional = True, dropout = 0.1):
        super(RNN, self).__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx = pad_idx)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers = no_of_layers, bidirectional = bidirectional, dropout = dropout)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, text, text_lengths):
        embedding = self.dropout(self.embedding(text))
        packed_embedding = nn.utils.rnn.pack_padded_sequence(embedding, text_lengths)
        packed_output, (hidden, cell) = self.rnn(packed_embedding)
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)
        output = self.dropout(torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim = 1))
        output = self.fc(output)
        return output

In [0]:
input_dim = len(TEXT.vocab)
embedding_dim = 100
hidden_dim = 256
output_dim = 1
pad_idx = TEXT.vocab.stoi[TEXT.pad_token]
dropout = 0.5
bidirectional = True
no_of_layers = 2

model = RNN(input_dim, embedding_dim, hidden_dim, output_dim, pad_idx, no_of_layers, bidirectional, dropout)

In [10]:
pretrained_embeddings = TEXT.vocab.vectors
print (pretrained_embeddings.shape)

torch.Size([25002, 100])


In [11]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-1.1507, -0.4920,  1.0079,  ..., -0.4377, -0.4171, -0.4464],
        [-1.1612, -1.1468,  0.6482,  ..., -0.7333, -0.2135,  0.6706],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 0.2550,  0.6494,  0.3156,  ..., -1.1001,  0.6163,  0.1800],
        [ 0.2564,  0.0799,  0.5741,  ..., -0.4184,  0.1494,  0.3029],
        [ 0.9388, -0.2669,  0.4541,  ...,  0.0668, -0.2694, -0.5749]])

In [0]:
unk_idx = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[unk_idx] = torch.zeros(embedding_dim)
model.embedding.weight.data[pad_idx] = torch.zeros(embedding_dim)

In [0]:
import torch.optim as optim
optimizer = optim.Adam(model.parameters())

In [0]:
criterion = nn.BCEWithLogitsLoss()

In [0]:
model = model.to(device)
criterion = criterion.to(device)

In [0]:
def binary_accuracy(preds, y):
    round_preds = torch.round(torch.sigmoid(preds))
    correct = (round_preds == y).float()
    acc = correct.sum() / len(correct)
    return acc

In [0]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_accuracy = 0
    model.train()
    model.zero_grad()
    for batch in iterator:
        text, text_lengths = batch.text
        predictions = model(text, text_lengths).squeeze(1)
        loss = criterion(predictions, batch.label)
        accuracy = binary_accuracy(predictions, batch.label)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 50)
        optimizer.step()

        epoch_loss += loss
        epoch_accuracy += accuracy

    return epoch_loss / len(iterator), epoch_accuracy / len(iterator) 

In [0]:
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_accuracy = 0
    
    model.eval()
    with torch.no_grad():
        for batch in iterator:
            text, text_lengths = batch.text
            predictions = model(text, text_lengths).squeeze(1)
            loss = criterion(predictions, batch.label)
            accuracy = binary_accuracy(predictions, batch.label)

            epoch_loss += loss
            epoch_accuracy += accuracy
        
    return epoch_loss / len(iterator), epoch_accuracy / len(iterator)

In [0]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [21]:
no_of_epochs = 50
best_valid_loss = float('inf')

for epoch in range(no_of_epochs):
    start_time = time.time()

    train_loss, train_accuracy = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_accuracy = evaluate(model, valid_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')

    print ('Epoch :  {} | time : {} mins {} secs'.format(epoch + 1, epoch_mins, epoch_secs))
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_accuracy*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_accuracy*100:.2f}%')

Epoch :  1 | time : 0 mins 43 secs
	Train Loss: 0.704 | Train Acc: 52.18%
	 Val. Loss: 0.677 |  Val. Acc: 53.13%
Epoch :  2 | time : 0 mins 44 secs
	Train Loss: 0.679 | Train Acc: 59.73%
	 Val. Loss: 0.643 |  Val. Acc: 59.83%
Epoch :  3 | time : 0 mins 44 secs
	Train Loss: 0.679 | Train Acc: 59.73%
	 Val. Loss: 0.640 |  Val. Acc: 62.82%
Epoch :  4 | time : 0 mins 45 secs
	Train Loss: 0.618 | Train Acc: 67.40%
	 Val. Loss: 0.691 |  Val. Acc: 59.90%
Epoch :  5 | time : 0 mins 45 secs
	Train Loss: 0.582 | Train Acc: 70.21%
	 Val. Loss: 0.551 |  Val. Acc: 73.65%
Epoch :  6 | time : 0 mins 45 secs
	Train Loss: 0.525 | Train Acc: 75.41%
	 Val. Loss: 0.503 |  Val. Acc: 75.32%
Epoch :  7 | time : 0 mins 45 secs
	Train Loss: 0.445 | Train Acc: 79.71%
	 Val. Loss: 0.392 |  Val. Acc: 83.13%
Epoch :  8 | time : 0 mins 45 secs
	Train Loss: 0.399 | Train Acc: 82.65%
	 Val. Loss: 0.385 |  Val. Acc: 83.12%
Epoch :  9 | time : 0 mins 45 secs
	Train Loss: 0.359 | Train Acc: 84.94%
	 Val. Loss: 0.349 |  

In [23]:
test_loss, test_accuracy = evaluate(model, test_iterator, criterion)
print ('The test loss is : {} and test accuracy is {}'.format(test_loss, test_accuracy))

The test loss is : 0.3525247871875763 and test accuracy is 0.85053551197052


In [0]:
import spacy
nlp = spacy.load('en')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return prediction.item()

In [42]:
predict_sentiment(model, 'This is film is garbage at best')

0.5037786960601807